In [1]:
import random
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import cv2
import warnings
import timeit
import pytorch_lightning as pl
from torch import optim
from scipy.special import expit
from skimage import io, transform, measure
from sklearn import metrics
import optuna
from config import *
from util import *
from models import *
sns.set()
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
rc = {"figure.figsize" : (9, 7),
      "axes.spines.left" : False,
      "axes.spines.right" : False,
      "axes.spines.bottom" : False,
      "axes.spines.top" : False,
      "xtick.bottom" : True,
      "xtick.labelbottom" : False,
      "ytick.labelleft" : False,
      "ytick.left" : True,
      "axes.grid" : False}
plt.rcParams.update(rc)

/home/caduser/anaconda3/envs/overlap/lib/python3.9/site-packages/neptune/internal/backends/hosted_client.py:50: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version
/home/caduser/anaconda3/envs/overlap/lib/python3.9/site-packages/pytorch_lightning/loggers/neptune.py:41: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  from neptune import new as neptune


In [ ]:
best_accuracy = 0

def objective(trial):
    torch.cuda.empty_cache()
    random.seed(seed)
    
    params = {
        #'epochs': trial.suggest_int('epochs', 10, 30),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
        'dx_dropout': trial.suggest_float('dx_dropout', 0.3, 0.7),
        'char_dropout': trial.suggest_float('char_dropout', 0.3, 0.7),
        'batch_size': trial.suggest_categorical('batch_size', [16, 32])
    }
    
    model = SCPPretrain(img_dir=img_dir, annotations_dir=annotations_dir, metadata_file=metadata_file, weighted_sampling=False,
                        batch_size=params['batch_size'], dx_dropout=params['dx_dropout'], char_dropout=params['char_dropout'],
                        learning_rate=params['learning_rate'])

    trainer = pl.Trainer(max_epochs=15, devices=1, accelerator="gpu", deterministic=True)
    trainer.fit(model)
    
    # Val predictions
    result_val = get_char_predictions(trainer, model, split='val', threshold=0)
    result_val['prediction'] = result_val['dx_pred'].apply(lambda x: 1 if x >= 0 else 0)
    thresholds = get_thresholds(result_val)
    dx_threshold = find_optimal_cutoff(result_val['benign_malignant'], result_val['dx_pred'])
    
    # Test predictions
    result_test = get_char_predictions(trainer, model, split='test', threshold=thresholds)
    result_test['prediction'] = get_diagnosis_predictions(result_test, num_melanoma_chars=1)
    result_test['prediction'] = result_test['dx_pred'].apply(lambda x: 1 if x >= dx_threshold else 0)
    
    target_model = HAMFineTune(img_dir=img_dir, annotations_dir=annotations_dir, metadata_file=metadata_file, weighted_sampling=False,
                                  batch_size=batch_size, dropout=dropout, learning_rate=learning_rate, char_pos_weight=char_pos_weight, dx_pos_weight=dx_pos_weight)

    target_model.load_state_dict(model.state_dict())
    trainer = pl.Trainer(max_epochs=20, devices=1, accelerator="gpu", deterministic=True)
    trainer.fit(target_model)

    val_accuracy = metrics.balanced_accuracy_score(result_val['benign_malignant'], result_val['prediction'])
    test_accuracy = metrics.balanced_accuracy_score(result_test['benign_malignant'], result_test['prediction'])
    
    trial.set_user_attr("val_accuracy", val_accuracy)
    trial.set_user_attr("test_accuracy", test_accuracy)
    
    return val_accuracy

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=seed), storage="sqlite:///optuna/pretrain.db")
study.optimize(objective, n_trials=15, gc_after_trial=True)